In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

#### 1. 将输入转化成TFRecord格式并保存。

In [2]:
# 定义函数转化变量类型。
# 生成整数型属性
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
# 生成字符串型属性
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 将数据转化为tf.train.Example格式。
def _make_example(pixels, label, image):
    image_raw = image.tostring()
    example = tf.train.Example(features=tf.train.Features(feature={
        'pixels': _int64_feature(pixels), # 训练数据的图像分辨率
        'label': _int64_feature(np.argmax(label)),
        'image_raw': _bytes_feature(image_raw)
    }))
    return example

# 读取mnist训练数据。
mnist = input_data.read_data_sets("../0_datasets/MNIST_data",dtype=tf.uint8, one_hot=True)
images = mnist.train.images
labels = mnist.train.labels
pixels = images.shape[1] # 训练数据的图像分辨率
num_examples = mnist.train.num_examples

# 输出包含训练数据的TFRecord文件。
with tf.python_io.TFRecordWriter("output.tfrecords") as writer: # wirter
    for index in range(num_examples):
        example = _make_example(pixels, labels[index], images[index])
        writer.write(example.SerializeToString())
print("TFRecord训练文件已保存。")

# 读取mnist测试数据。
images_test = mnist.test.images
labels_test = mnist.test.labels
pixels_test = images_test.shape[1]
num_examples_test = mnist.test.num_examples

# 输出包含测试数据的TFRecord文件。
with tf.python_io.TFRecordWriter("output_test.tfrecords") as writer:
    for index in range(num_examples_test):
        example = _make_example(
            pixels_test, labels_test[index], images_test[index])
        writer.write(example.SerializeToString())
print("TFRecord测试文件已保存。")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../0_datasets/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../0_datasets/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../0_datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../0_datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
TFRecord训练文件已保存。
TFRecord测试文件已保存。


#### 2. 读取TFRecord文件
* tf.train.string_input_producer

In [3]:
# 读取文件。
reader = tf.TFRecordReader()
# 创建一个队列来维护输入文件列表。
filename_queue = tf.train.string_input_producer(["output.tfrecords"]) #　train.tfrecords
# 从文件中读取一个样例，也可以使用`read_up_to()`一次性读取多个样例。
_,serialized_example = reader.read(filename_queue)

# 解析读取的一个样例。解析多个样例，使用`parse_example()`函数。
features = tf.parse_single_example(
    serialized_example,
    features={
        # tf.FixedLenFeature解析为一个tensor;
        # tf.VarLenFeature解析为SparseTensor,用于处理稀疏数据。
        # 解析时的格式要和写入tfrecords时一致。
        'image_raw':tf.FixedLenFeature([],tf.string),
        'pixels':tf.FixedLenFeature([],tf.int64),
        'label':tf.FixedLenFeature([],tf.int64)
    })
# tf.decode_raw将字符串解析为图像的像素数组。
images = tf.decode_raw(features['image_raw'],tf.uint8)
labels = tf.cast(features['label'],tf.int32)
pixels = tf.cast(features['pixels'],tf.int32)
# 会话
sess = tf.Session()

# 启动多线程处理输入数据。
coord = tf.train.Coordinator() # 该类实现了一种简单的机制来协调一组线程的终止。
threads = tf.train.start_queue_runners(sess=sess,coord=coord)
# 每次运行可以读取TFRecords文件中的一个example,所有样例读取完后，在此样例中程序会重头读取。
for i in range(10):
    image, label, pixel = sess.run([images, labels, pixels])